In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import os

In [13]:
with open('quijote.txt', 'r') as f:
    text = f.read()

print(text[:300])

﻿El ingenioso hidalgo don Quijote de la Mancha


TASA

Yo, Juan Gallo de Andrada, escribano de Cámara del Rey nuestro señor, de
los que residen en su Consejo, certifico y doy fe que, habiendo visto por
los señores dél un libro intitulado El ingenioso hidalgo de la Mancha,
compuesto por Miguel de Cer


In [14]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])
encoded[:100]

array([62, 15, 39, 80, 13, 56, 53,  1, 56, 13, 72, 81, 72, 80, 59, 13, 67,
       75, 39, 53, 72, 80, 67, 72, 56, 80, 82, 86, 13,  2, 72, 47,  1, 80,
       67,  1, 80, 39, 75, 80, 87, 75, 56,  7, 59, 75, 90, 90, 90, 79, 46,
       84, 46, 90, 90, 76, 72, 54, 80, 55, 86, 75, 56, 80, 89, 75, 39, 39,
       72, 80, 67,  1, 80, 46, 56, 67, 66, 75, 67, 75, 54, 80,  1, 81,  7,
       66, 13, 38, 75, 56, 72, 80, 67,  1, 80, 45, 18, 91, 75, 66])

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [16]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make, // integer division, round down
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows, n. of first row is the batch size, the other lenght is inferred
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y 
        
#when we call get batches we are going 
#to create a generator that iteratest through our array and returns x, y with yield command

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [19]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[62 15 39 80 13 56 53  1 56 13]
 [91 75 39 80 20 66  1 81 63 72]
 [54 80  8 86  1 80 47 34 80  1]
 [80 91 13 80 81  1 64 72 66 54]
 [19 39 80 59 75 38 39 65 80 67]
 [81 47 66  1 21 75 80 91 72 67]
 [29 80 47  1 80  1 56 85 29 72]
 [ 1 66 72 90 47 72 67 75 81 80]]

y
 [[15 39 80 13 56 53  1 56 13 72]
 [75 39 80 20 66  1 81 63 72 56]
 [80  8 86  1 80 47 34 80  1 66]
 [91 13 80 81  1 64 72 66 54 80]
 [39 80 59 75 38 39 65 80 67  1]
 [47 66  1 21 75 80 91 72 67  1]
 [80 47  1 80  1 56 85 29 72 80]
 [66 72 90 47 72 67 75 81 80 81]]


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=612, n_layers=4,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
       
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [21]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [23]:
# define and print the net
n_hidden=512
n_layers=4

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(92, 512, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=92, bias=True)
)


In [24]:
batch_size = 64
seq_length = 160 #max length verses
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 3.2026... Val Loss: 3.1315
Epoch: 1/50... Step: 20... Loss: 3.1107... Val Loss: 3.0679
Epoch: 1/50... Step: 30... Loss: 3.0902... Val Loss: 3.0600
Epoch: 1/50... Step: 40... Loss: 3.0724... Val Loss: 3.0591
Epoch: 1/50... Step: 50... Loss: 3.0702... Val Loss: 3.0572
Epoch: 1/50... Step: 60... Loss: 3.0689... Val Loss: 3.0560
Epoch: 1/50... Step: 70... Loss: 3.0505... Val Loss: 3.0576
Epoch: 1/50... Step: 80... Loss: 3.0441... Val Loss: 3.0565
Epoch: 1/50... Step: 90... Loss: 3.0470... Val Loss: 3.0563
Epoch: 1/50... Step: 100... Loss: 3.0306... Val Loss: 3.0555
Epoch: 1/50... Step: 110... Loss: 3.0516... Val Loss: 3.0566
Epoch: 1/50... Step: 120... Loss: 3.0717... Val Loss: 3.0558
Epoch: 1/50... Step: 130... Loss: 3.0502... Val Loss: 3.0550
Epoch: 1/50... Step: 140... Loss: 3.0559... Val Loss: 3.0548
Epoch: 1/50... Step: 150... Loss: 3.0371... Val Loss: 3.0554
Epoch: 1/50... Step: 160... Loss: 3.0716... Val Loss: 3.0548
Epoch: 1/50... Step: 170... Loss:

In [0]:
model_dante = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_dante, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        # apply softmax to get p probabilities for the likely next character giving x
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        # considering the k most probable characters with topk method
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='Il', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [32]:
print(sample(net, 1000, prime='Los ', top_k=5))


Los a la mano, que
sería de parecer, sino a su carricela, como yo se le vio a la collida a la vida, y
le dijo:

-No me habéis dado los cuatro de sus culpas que no los son después de la cual lana
cuando el caballo de mi amo. Esté de muy mal es el primero que tienes caballero
andante; y así, estáis al cielo, con la vida que se las estuviera para que le
presentares y se entiende que esta veces las muchas carreras ha de hacer esta señora lugar
de mi amo que alguna venta la dice y descubre a las ardas a todos los de los demás que
dicen, y esto no hiredes y descriidos, que las provechosas
parecen como es lo que se halla y asentará en el cielo destos
caballeros a don Quijote que no pasasen la vara y dispuesta de sus poetas. Desde esta parte de mucho
desalmado día la duelme, y ella no he hacedo cerrado de la cera, y después de ser
de mañana; y no es ella es mal a las aventuras que satisfechos pasado al duque que
el caballo de las circunstancias desposaban de caballero en la mejor que a los
que

In [33]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [35]:
# Sample using a loaded model
print(sample(loaded, 800, top_k=5, prime="El mejor amigo "))

El mejor amigo que los caballeros caballeros andantes
posible, allí se despierta en la corte para perder el amor de su
ampiente, de la prisión de mi puerta por las muchas que se pusieron el cobordado.

-Nunca la debe de llamar en verdad -respondió el barbero-, que en tiempo, y que no se
encogieran es el mismo delicado, porque estas muderas que le tengon cortado y el
primero que se podrá ser después de lo que está en el señor don Quijote de
su camino, que sea encantado al cielo espuela, y no sé el capitán: yo soy era que
sea, así de la mano, que soy mucha parte, y de todos estos dos los cuales de
caballeros andantes sacaron a sus propias. Ellas estuviérades la profesión de la cena, porque
se he visto a todas las palmas de camino el de capecen, que nos pensé yo decir: que en
la mitad de locas el caballo, qu
